In [5]:
import time
import numpy as np
from openmm import app
import openmm as mm
from openmm import unit as u
import pdbfixer
from sys import stdout

In [6]:
STRIDE = 100
simulations_steps = 10000
temperature = 300 * u.kelvin
platform_name = 'CPU'#'OpenCL' 
real_time_STRIDE = 1000
friction_coeff = 1.0/u.picoseconds
time_step = 2.0*u.femtoseconds

In [7]:
# Loading the file into the notebook.
pdb_file = 'data/peptide_fixed.pdb'
fixer = pdbfixer.PDBFixer(pdb_file)

In [8]:
# Defining the forcfield and the model
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
model = app.Modeller(fixer.topology, fixer.positions)
model.addSolvent(forcefield,
             padding=1.*u.nanometer,
             #boxSize=mm.Vec3(3.0, 3.0, 3.0) * u.nanometers ,
             model='tip3p', 
             ionicStrength=0.1 * u.millimolar, 
             positiveIon='Na+',
             negativeIon='Cl-',
             neutralize=True )

app.PDBFile.writeFile(model.topology,model.positions, open('data/peptide_model_water.pdb', 'w'))###add

system = forcefield.createSystem(model.topology, 
                                 nonbondedMethod=app.PME, 
                                 nonbondedCutoff=1*u.nanometers, 
                                 constraints=app.HBonds, 
                                 rigidWater=True, 
                                 ewaldErrorTolerance=0.0005)

integrator = mm.LangevinIntegrator(temperature,
                                   friction_coeff,
                                   time_step)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName(platform_name)
simulation = app.Simulation(model.topology, system, integrator, platform)
simulation.context.setPositions(model.positions)

#start = time.time()###delete
#simulation.reporters.append(mm.app.pdbreporter.PDBReporter('data/output.pdb', STRIDE))###delete
simulation.reporters.append(mm.app.dcdreporter.DCDReporter('data/output.dcd', STRIDE))###add
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter('data/mm.log', STRIDE, step=True, potentialEnergy=True, temperature=True, time=True))
simulation.reporters.append(mm.app.statedatareporter.StateDataReporter(stdout, real_time_STRIDE, step=True, potentialEnergy=True, temperature=True, speed=True, elapsedTime=True))
simulation.minimizeEnergy()
simulation.step(simulations_steps)
#end = time.time()

#print(f"Elapsed real time: {end-start} s")###delete
#print(f'I could do {(60*60)/(end-start)*simulations_steps} simulation steps per hour with {simulations_steps/STRIDE} frames saved.')###delete

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Elapsed Time (s)"
1000,-134267.12625187865,256.13575903494393,--,0.0
2000,-130524.08068469961,292.59004894587883,24,7.200934886932373
3000,-130027.02030057537,295.11462986894503,24,14.404328346252441
4000,-129010.83041556443,296.8981224214023,24.2,21.4354031085968
5000,-129220.964384086,305.187713554717,24.2,28.52867865562439
6000,-128511.93558622336,296.8530789239235,24.1,35.81447792053223
7000,-128723.76948960451,302.94286566918197,24.1,42.94683480262756
8000,-128533.38013831215,301.8911240714617,24.1,50.25991153717041
9000,-129386.66332067773,300.62941275795373,24,57.62273454666138
10000,-129773.39655592231,301.5721362388712,23.9,65.04771733283997
